## 연간 엔터사별 Top 앨범 출시에 따른 주가 변동률 분석

- Top 앨범 출시일 이후 10 거래일 간의 주가 변동률 분석
- 주가 변동률 분석 함수: calculation_of_stock_by_release_date 

- 일간 변동률 = daily rate of change (drc)
- 변동률 누적 합계 = cumulative sum rate of change (cs)

### 1. 특정 앨범 출시에 따른 주가 변동률 분석

In [68]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
from calculation_of_stock_by_release_date import create_top_album_stock, calculate_rate_of_change

In [69]:
yearly_top_album_df = pd.read_csv('data/yearly_top_album_df.csv')

- 2018년도 SM Top 앨범 출시에 따른 주가 변동률 분석

In [70]:
yearly_top_album_df.loc[0]

agency_name                                         SM
year                                              2018
ranking                                              3
singer                                             EXO
title           DON'T MESS UP MY TEMPO - The 5th Album
sales                                          1452030
release_date                                2018.11.02
Name: 0, dtype: object

In [71]:
# 분석할 출시일
release_date = yearly_top_album_df.loc[0][6]

# Top 앨범 출시일 이후 10 거래일 간의 주가 데이터 추출
conn = pymysql.connect(host='127.0.0.1'
            , port=3306
            , user = 'root'
            , password='root1234'
            , db = 'stock_pj'
            , charset='utf8'
            ) 

cur = conn.cursor()

cur.execute(f'''
    select * from agency_stock
    where date >= '{release_date}'
    ORDER BY date
    LIMIT 30;   
''')

top_album_stock_tuple = cur.fetchall()

# create_top_album_stock
top_album_stock_df, date_list = create_top_album_stock(top_album_stock_tuple)

# 주가 변동률 계산
rate_of_change_df = calculate_rate_of_change(top_album_stock_df, date_list)
rate_of_change_df

,date,jyp_drc,jyp_cs,sm_drc,sm_cs,yg_drc,yg_cs
0,2018-11-02,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-11-05,-3.278689,-3.278689,1.656315,1.656315,-3.728362,-3.728362
2,2018-11-06,-0.993377,-4.272066,-2.006336,-0.350021,-1.899593,-5.627955
3,2018-11-07,-3.247863,-7.519929,-1.827957,-2.177978,-3.802817,-9.430772
4,2018-11-08,-1.916376,-9.436306,-1.861993,-4.039972,0.560224,-8.870548
5,2018-11-09,3.529412,-5.906894,2.143623,-1.896349,1.381215,-7.489332
6,2018-11-12,4.186795,-1.720098,0.107066,-1.789282,5.359477,-2.129855
7,2018-11-13,-0.811688,-2.531787,-0.430108,-2.219390,4.015056,1.885201
8,2018-11-14,5.521472,2.989686,6.813627,4.594237,3.510896,5.396097
9,2018-11-15,5.232558,8.222244,7.249071,11.843308,2.364066,7.760163


분석 결과)
* JYP 8.22%, SM 11.84%, yg 7.76%
* 2018년도 SM의 Top 앨범 출시일에 따른 주가 변동성은 SM이 제일 높다

----

### 2. 5년 간 엔터사별 Top 앨범 출시에 따른 주가 변동률 분석

In [72]:
agency_name_list = []
year_list = []
jyp_cs_list = []
sm_cs_list = []
yg_cs_list = []

for i in range(len(yearly_top_album_df)):
    agency_name_list.append(yearly_top_album_df['agency_name'][i])
    year_list.append(yearly_top_album_df['year'][i])
    release_date = yearly_top_album_df['release_date'][i]

    # Top 앨범 출시일 이후 10 거래일 간의 주가 데이터 추출
    conn = pymysql.connect(host='127.0.0.1'
            , port=3306
            , user = 'root'
            , password='root1234'
            , db = 'stock_pj'
            , charset='utf8'
            ) 

    cur = conn.cursor()

    cur.execute(f'''
        select * from agency_stock
        where date >= '{release_date}'
        ORDER BY date
        LIMIT 30;   
    ''')

    top_album_stock_tuple = cur.fetchall()

    # create_top_album_stock
    top_album_stock_df, date_list = create_top_album_stock(top_album_stock_tuple)

    # 주가 변동률 계산
    rate_of_change_df = calculate_rate_of_change(top_album_stock_df, date_list)

    # 엔터사별 누적 주가 변동률 추가
    jyp_cs_list.append(rate_of_change_df.loc[9, 'jyp_cs'])                                                  
    sm_cs_list.append(rate_of_change_df.loc[9, 'sm_cs'])                                                  
    yg_cs_list.append(rate_of_change_df.loc[9, 'yg_cs'])                                                  


cs_df = pd.DataFrame()
cs_df['agency_name'] = agency_name_list
cs_df['year'] = year_list
cs_df['jyp_cs'] = jyp_cs_list
cs_df['sm_cs'] = sm_cs_list
cs_df['yg_cs'] = yg_cs_list
cs_df.shape

(15, 5)

In [73]:
# 주가 변동률 계산
cs_df

,agency_name,year,jyp_cs,sm_cs,yg_cs
0,SM,2018,8.222244,11.843308,7.760163
1,JYP,2018,1.071783,-9.207872,-0.964885
2,YG,2018,2.230056,1.186451,14.925796
3,SM,2019,-2.541081,-4.961428,2.215234
4,JYP,2019,-3.525894,-5.555336,0.528749
5,YG,2019,-6.245536,1.654020,-1.835430
6,SM,2020,-6.775966,-12.561541,-12.988525
7,YG,2020,-8.850483,-11.260996,-21.287346
8,JYP,2020,-7.897990,-5.566956,-0.508599
9,SM,2021,6.036600,17.440384,16.578404


* 다른 두 회사의 주가 변동률에 비해 기준 회사의 주가 변동률이 크면 True, 아니면 False

In [74]:
# 둘 중 하나 보다 크면 True
agency_idx = {'JYP':2, 'SM':3, 'YG':4}
true_false_list = []

for i in range(len(cs_df)):
    idx_list = list(agency_idx.values())
    idx = agency_idx[cs_df.loc[i][0]]
    idx_list.remove(idx)
    true_false_list.append((cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[0]]) or (cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[1]])) # or

cs_df['result'] = true_false_list
cs_df

,agency_name,year,jyp_cs,sm_cs,yg_cs,result
0,SM,2018,8.222244,11.843308,7.760163,True
1,JYP,2018,1.071783,-9.207872,-0.964885,True
2,YG,2018,2.230056,1.186451,14.925796,True
3,SM,2019,-2.541081,-4.961428,2.215234,False
4,JYP,2019,-3.525894,-5.555336,0.528749,True
5,YG,2019,-6.245536,1.654020,-1.835430,True
6,SM,2020,-6.775966,-12.561541,-12.988525,True
7,YG,2020,-8.850483,-11.260996,-21.287346,False
8,JYP,2020,-7.897990,-5.566956,-0.508599,False
9,SM,2021,6.036600,17.440384,16.578404,True


In [75]:
# True 비율
true_cnt = cs_df.loc[cs_df['result'] == True, 'result'].count()
false_cnt = cs_df.loc[cs_df['result'] == False, 'result'].count()
print(f"True: {true_cnt}")
print(f"False: {false_cnt}")
print(f"True 비율: {true_cnt/15 * 100} %")

True: 9
False: 6
True 비율: 60.0 %


In [76]:
# 두 회사보다 모두 크면 True
agency_idx = {'JYP':2, 'SM':3, 'YG':4}
true_false_list = []

for i in range(len(cs_df)):
    idx_list = list(agency_idx.values())
    idx = agency_idx[cs_df.loc[i][0]]
    idx_list.remove(idx)
    true_false_list.append((cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[0]]) and (cs_df.loc[i][idx] >= cs_df.loc[i][idx_list[1]])) # and 

cs_df['result'] = true_false_list

true_cnt = cs_df.loc[cs_df['result'] == True, 'result'].count()
false_cnt = cs_df.loc[cs_df['result'] == False, 'result'].count()
print(f"True: {true_cnt}")
print(f"False: {false_cnt}")
print(f"True 비율: {true_cnt/15 * 100} %")

True: 5
False: 10
True 비율: 33.33333333333333 %


최종 분석 결과)

* 주가 변동률이 두 회사보다 모두 큰 경우는 약 33% 정도로 낮다. 그러므로, 앨범 출시와 엔터사 주식과는 큰 관계가 없어 가설을 입증할 수 없다. <br>
또한, 분석 데이터로 보아 2020년 YG의 주가 변동률이 매우 높은 것으로 보이는데 이는 해당 날짜의 소속사에 큰 논란이 있었던 것으로 확인된다. <br>
때문에, 소속사에 관한 긍정/부정적인 기사와 주가와의 연관성을 분석해보는 것도 좋을 것 같다.

----

In [20]:
## 절댓값

# 둘 중 하나 보다 크면 True
agency_idx = {'JYP':2, 'SM':3, 'YG':4}
true_false_list = []

for i in range(len(cs_df)):
    idx_list = list(agency_idx.values())
    idx = agency_idx[cs_df.loc[i][0]]
    idx_list.remove(idx)
    true_false_list.append((abs(cs_df.loc[i][idx]) >= abs(cs_df.loc[i][idx_list[0]])) or (abs(cs_df.loc[i][idx]) >= abs(cs_df.loc[i][idx_list[1]]))) # or

cs_df['result'] = true_false_list
cs_df

,agency_name,year,jyp_cs,sm_cs,yg_cs,result
0,SM,2018,8.222244,11.843308,7.760163,True
1,JYP,2018,1.071783,-9.207872,-0.964885,True
2,YG,2018,2.230056,1.186451,14.925796,True
3,SM,2019,-2.541081,-4.961428,2.215234,True
4,JYP,2019,-3.525894,-5.555336,0.528749,True
5,YG,2019,-6.245536,1.654020,-1.835430,True
6,SM,2020,-6.775966,-12.561541,-12.988525,True
7,YG,2020,-8.850483,-11.260996,-21.287346,True
8,JYP,2020,-7.897990,-5.566956,-0.508599,True
9,SM,2021,6.036600,17.440384,16.578404,True


In [21]:
# True 비율
true_cnt = cs_df.loc[cs_df['result'] == True, 'result'].count()
false_cnt = cs_df.loc[cs_df['result'] == False, 'result'].count()
print(f"True: {true_cnt}")
print(f"False: {false_cnt}")
print(f"True 비율: {true_cnt/15 * 100} %")

True: 13
False: 2
True 비율: 86.66666666666667 %


In [24]:
# 두 회사보다 모두 크면 True
agency_idx = {'JYP':2, 'SM':3, 'YG':4}
true_false_list = []

for i in range(len(cs_df)):
    idx_list = list(agency_idx.values())
    idx = agency_idx[cs_df.loc[i][0]]
    idx_list.remove(idx)
    true_false_list.append((abs(cs_df.loc[i][idx]) >= abs(cs_df.loc[i][idx_list[0]])) and (abs(cs_df.loc[i][idx]) >= abs(cs_df.loc[i][idx_list[1]]))) # and 

cs_df['result'] = true_false_list

cs_df

,agency_name,year,jyp_cs,sm_cs,yg_cs,result
0,SM,2018,8.222244,11.843308,7.760163,True
1,JYP,2018,1.071783,-9.207872,-0.964885,False
2,YG,2018,2.230056,1.186451,14.925796,True
3,SM,2019,-2.541081,-4.961428,2.215234,True
4,JYP,2019,-3.525894,-5.555336,0.528749,False
5,YG,2019,-6.245536,1.654020,-1.835430,False
6,SM,2020,-6.775966,-12.561541,-12.988525,False
7,YG,2020,-8.850483,-11.260996,-21.287346,True
8,JYP,2020,-7.897990,-5.566956,-0.508599,True
9,SM,2021,6.036600,17.440384,16.578404,True


In [ ]:
true_cnt = cs_df.loc[cs_df['result'] == True, 'result'].count()
false_cnt = cs_df.loc[cs_df['result'] == False, 'result'].count()
print(f"True: {true_cnt}")
print(f"False: {false_cnt}")
print(f"True 비율: {true_cnt/15 * 100} %")